In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from mpl_toolkits.mplot3d import axes3d

In [2]:
filename_in_dir = []

for root, dirs, files in os.walk('swing'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print(filename_in_dir)

['swing/fo_drive/EXER_20210107_152154.csv', 'swing/fo_drive/EXER_20210105_102123.csv', 'swing/fo_drive/EXER_20210105_102110.csv', 'swing/fo_drive/EXER_20210105_102106.csv', 'swing/fo_drive/EXER_20210105_102046.csv', 'swing/fo_drive/EXER_20210107_152238.csv', 'swing/fo_drive/EXER_20210105_102147.csv', 'swing/fo_drive/EXER_20210105_101947.csv', 'swing/fo_drive/EXER_20210105_102200.csv', 'swing/fo_drive/EXER_20210105_102149.csv', 'swing/fo_drive/EXER_20210107_152150.csv', 'swing/fo_drive/EXER_20210105_102204.csv', 'swing/fo_drive/EXER_20210107_152102.csv', 'swing/fo_drive/EXER_20210107_152220.csv', 'swing/fo_drive/EXER_20210107_152341.csv', 'swing/fo_drive/EXER_20210107_152055.csv', 'swing/fo_drive/EXER_20210105_102049.csv', 'swing/fo_drive/EXER_20210107_152244.csv', 'swing/fo_drive/EXER_20210105_102116.csv', 'swing/fo_drive/EXER_20210107_152338.csv', 'swing/fo_drive/EXER_20210105_102003.csv', 'swing/fo_drive/EXER_20210105_102209.csv', 'swing/fo_drive/EXER_20210105_102141.csv', 'swing/fo_

In [3]:
total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)



In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

In [5]:
from sklearn.model_selection import train_test_split
total_data = np.array(total_data)
arr_label = np.array(arr_label)
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)


In [6]:
from tensorflow.keras.layers import Dropout,Input,Dense,Activation,Flatten,SeparableConv2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.8 
session = tf.Session(config=config)


/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dty

1 Physical GPUs, 1 Logical GPUs


In [7]:
from tensorflow.keras.layers import LSTM,Conv1D,MaxPooling1D
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
import tensorflow.keras.backend as K 
from tensorflow.keras.callbacks import EarlyStopping

with tf.device('/GPU:1'):
    model = Sequential() # Sequeatial Model 
    model.add(LSTM(60, input_shape=(60,3),return_sequences = True)) # (timestep, feature) 
    model.add(Dropout(0.2))
    model.add(Conv1D(256,
                     2,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(128))
    model.add(Dense(8, activation='softmax'))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist = model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 1328 samples, validate on 332 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
1328/1328 [==============================] - 3s 2ms/sample - loss: 1.3511 - acc: 0.5407 - val_loss: 0.5598 - val_acc: 0.8313
Epoch 2/100
1328/1328 [==============================] - 1s 905us/sample - loss: 0.3583 - acc: 0.8908 - val_loss: 0.2136 - val_acc: 0.9458
Epoch 3/100
1328/1328 [==============================] - 1s 898us/sample - loss: 0.2267 - acc: 0.9270 - val_loss: 0.1891 - val_acc: 0.9458
Epoch 4/100
1328/1328 [==============================] - 1s 878us/sample - loss: 0.1715 - acc: 0.9473 - val_loss: 0.1356 - val_acc: 0.9699
Epoch 5/100
1328/1328 [==============================] - 1s 876us/sample - loss: 0.1157 - acc: 0.9639 - val_loss: 0.1169 - val_acc: 0.9699
Epoch 6/100
1328/1328 [=========================

1328/1328 [==============================] - 1s 882us/sample - loss: 0.0048 - acc: 0.9985 - val_loss: 0.0619 - val_acc: 0.9880
Epoch 55/100
1328/1328 [==============================] - 1s 889us/sample - loss: 0.0023 - acc: 1.0000 - val_loss: 0.0608 - val_acc: 0.9880
Epoch 56/100
1328/1328 [==============================] - 1s 887us/sample - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0629 - val_acc: 0.9880
Epoch 57/100
1328/1328 [==============================] - 1s 880us/sample - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0637 - val_acc: 0.9880
Epoch 58/100
1328/1328 [==============================] - 1s 881us/sample - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0664 - val_acc: 0.9880
Epoch 59/100
1328/1328 [==============================] - 1s 895us/sample - loss: 0.0010 - acc: 1.0000 - val_loss: 0.0666 - val_acc: 0.9880
Epoch 60/100
1328/1328 [==============================] - 1s 886us/sample - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0685 - val_acc: 0.9849
Epoch 61/100
1328/1328 [=========

In [10]:
filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
#sample = total_data.reshape((nsamples,nx*ny))

sample_pred = model.predict(total_data)
sample_pred = np.argmax(sample_pred,axis=-1)
lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

testing new data result :
[answer]  -->  [predict err]
back_drive  -->  back_smash err_index :  16
back_smash  -->  fo_drive err_index :  73
back_smash  -->  fo_drive err_index :  74
back_smash  -->  fo_drive err_index :  75
back_smash  -->  fo_drive err_index :  77
back_smash  -->  fo_drive err_index :  84
back_smash  -->  fo_drive err_index :  85
back_smash  -->  fo_drive err_index :  86
hit:  79  miss :  8
